In [366]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [367]:
df = pd.read_csv('../data/data.tsv', sep='\t',error_bad_lines=False)

b'Skipping line 83032: expected 6 fields, saw 7\n'
b'Skipping line 154657: expected 6 fields, saw 7\n'
b'Skipping line 323916: expected 6 fields, saw 7\n'
/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [368]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,133273,213221,213222.0,How is the life of a math student? Could you d...,Which level of prepration is enough for the ex...,0.0
1,402555,536040,536041.0,How do I control my horny emotions?,How do you control your horniness?,1.0
2,360472,364011,490273.0,What causes stool color to change to yellow?,What can cause stool to come out as little balls?,0.0
3,150662,155721,7256.0,What can one do after MBBS?,What do i do after my MBBS ?,1.0
4,183004,279958,279959.0,Where can I find a power outlet for my laptop ...,"Would a second airport in Sydney, Australia be...",0.0


In [369]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363192 entries, 0 to 363191
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            363192 non-null  object 
 1   qid1          363192 non-null  object 
 2   qid2          363185 non-null  float64
 3   question1     363181 non-null  object 
 4   question2     363180 non-null  object 
 5   is_duplicate  363180 non-null  float64
dtypes: float64(2), object(4)
memory usage: 16.6+ MB


In [370]:
df = df.dropna(axis=0, subset=('question1','question2','is_duplicate' ))

In [371]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 363177 entries, 0 to 363191
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            363177 non-null  object 
 1   qid1          363177 non-null  object 
 2   qid2          363177 non-null  float64
 3   question1     363177 non-null  object 
 4   question2     363177 non-null  object 
 5   is_duplicate  363177 non-null  float64
dtypes: float64(2), object(4)
memory usage: 19.4+ MB


In [372]:
#lower casing
def lower_case(text):
    return text.lower()


#remove numbers
def remove_numbers(text):
    output = re.sub(r'\d+', '', text)
    return output

# remove punctuation
import string
def remove_punctuation(text):
    text_p = "".join([char for char in text if char not in string.punctuation])
    return text_p

#tokenize text

import nltk
nltk.download('punkt')
from nltk import word_tokenize

def tokenize(text):
    words = word_tokenize(text)
    return words

#remove stopwords
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

def remove_stopwords(text):
    filtered_words = [word for word in text if word not in stop_words]
    return filtered_words

#stemming
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def stem_words(text):
    stemmed = [stemmer.stem(word) for word in text]
    return stemmed

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [373]:
class Preprocessor():
    def __init__(self):
        pass
    
    def preprocess(self,df_column, steps):
        self.df_column = df_column
        self.steps = steps
    
        if 'lower_case' in self.steps:
            self.df_column = self.df_column.apply(lambda x: lower_case(x))
        
        
        if 'remove_numbers' in self.steps:
            self.df_column = self.df_column.apply(lambda x: remove_numbers(x))
        
        if 'remove_punctuation' in self.steps:
            self.df_column = self.df_column.apply(lambda x: remove_punctuation(x))           
        
        if 'tokenize' in self.steps:
            self.df_column = self.df_column.apply(lambda x: tokenize(x))
        
        if 'stopwords' in self.steps:
            self.df_column = self.df_column.apply(lambda x: remove_stopwords(x))
        
        if 'stemming' in self.steps:
            self.df_column = self.df_column.apply(lambda x: stem_words(x))
            
        return self.df_column    

In [374]:
steps = ['lower_case','remove_numbers','remove_punctuation',
        'tokenize','stopwords','stemming']
processor = Preprocessor()
df['question1'] = processor.preprocess(df['question1'] ,steps)

In [375]:
df['question2'] = processor.preprocess(df['question2'] ,steps)

In [376]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 363177 entries, 0 to 363191
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            363177 non-null  object 
 1   qid1          363177 non-null  object 
 2   qid2          363177 non-null  float64
 3   question1     363177 non-null  object 
 4   question2     363177 non-null  object 
 5   is_duplicate  363177 non-null  float64
dtypes: float64(2), object(4)
memory usage: 19.4+ MB


In [377]:
df.reset_index(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363177 entries, 0 to 363176
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   index         363177 non-null  int64  
 1   id            363177 non-null  object 
 2   qid1          363177 non-null  object 
 3   qid2          363177 non-null  float64
 4   question1     363177 non-null  object 
 5   question2     363177 non-null  object 
 6   is_duplicate  363177 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 19.4+ MB


In [378]:
df.head()

,index,id,qid1,qid2,question1,question2,is_duplicate
0,0,133273,213221,213222.0,"[life, math, student, could, describ, experi]","[level, preprat, enough, exam, jlpt]",0.0
1,1,402555,536040,536041.0,"[control, horni, emot]","[control, horni]",1.0
2,2,360472,364011,490273.0,"[caus, stool, color, chang, yellow]","[caus, stool, come, littl, ball]",0.0
3,3,150662,155721,7256.0,"[one, mbb]",[mbb],1.0
4,4,183004,279958,279959.0,"[find, power, outlet, laptop, melbourn, airport]","[would, second, airport, sydney, australia, ne...",0.0


In [379]:
def calc_tf(alist):
    
    tf_list = [alist.count(term)/len(alist) for term in alist]
    return tf_list


In [380]:
df['q2_tf'] = df['question2'].apply(lambda x: calc_tf(x))

In [381]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363177 entries, 0 to 363176
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   index         363177 non-null  int64  
 1   id            363177 non-null  object 
 2   qid1          363177 non-null  object 
 3   qid2          363177 non-null  float64
 4   question1     363177 non-null  object 
 5   question2     363177 non-null  object 
 6   is_duplicate  363177 non-null  float64
 7   q2_tf         363177 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 22.2+ MB


In [382]:
def get_DF(vocabulary_list,dataframe,tokens_col_name, id_col_name):
    DF = {}      #initialise empty document frequency dictionary
    
    #for each word in vocabulary, create key:value pair, with an empty list as value
    for word in vocabulary_list:
        DF[word] = [] 
        
    #for each word in DF, add the id of question2 where it appears. Create a set to avoid duplicate ids
    for i in range(len(dataframe)):
        for word in dataframe.loc[i,tokens_col_name]:
            DF[word].append(dataframe[id_col_name][i])
    for word in DF.keys():
        DF[word] = list(set(DF[word]))
    
    return DF


In [383]:
q2vocab = df['question2'].tolist()
q2vocab = [item for sublist in q2vocab for item in sublist]
q2vocab = list(set(q2vocab))
DF = get_DF(q2vocab,df,'question2','qid2')
DF

{'chromogen': [394623.0],
 'mentorcuminvestor': [429883.0],
 'stragetig': [388942.0],
 'emants': [482420.0],
 'adjoint': [316956.0],
 'uneducatedcomput': [331580.0],
 'textual': [504596.0],
 'cohort': [22881.0,
  236995.0,
  316585.0,
  87146.0,
  282389.0,
  101813.0,
  115802.0],
 'doga': [3354.0, 60835.0],
 'mna': [353868.0],
 'incest': [296972.0,
  69653.0,
  227862.0,
  119578.0,
  119579.0,
  5786.0,
  449312.0,
  470688.0,
  346402.0,
  808.0,
  809.0,
  342061.0,
  410031.0,
  468018.0,
  42930.0,
  489408.0,
  219329.0,
  125264.0,
  57428.0,
  57429.0,
  187611.0,
  477278.0,
  8159.0,
  8160.0,
  464097.0,
  81121.0,
  331744.0,
  361703.0,
  285290.0,
  471274.0,
  146544.0,
  452467.0,
  185717.0,
  58744.0,
  135295.0],
 'beginbmatrix': [237182.0],
 'supporttechn': [256957.0],
 'psuedaphedrin': [11523.0],
 'jpa': [227612.0],
 'internscoop': [385375.0],
 'yottabyt': [156186.0],
 'ltv': [362404.0, 236995.0, 89740.0],
 'samson': [515439.0],
 'sql': [210439.0,
  297996.0,
  3

In [365]:
len(DF)

50127